In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

1.13.1
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()

In [4]:
x_train_all.shape

(60000, 28, 28)

In [5]:
x_test.shape

(10000, 28, 28)

In [6]:
y_train_all.shape

(60000,)

In [7]:
x_valid,x_train =  x_train_all[:5000],x_train_all[5000:]
y_valid,y_train =  y_train_all[:5000],y_train_all[5000:]

In [8]:
x_train.shape

(55000, 28, 28)

In [9]:
np.max(x_train),np.min(x_train)

(255, 0)

In [10]:
# 对数据做标准化
# x = (x - u)/std (u:均值,std:方差)

from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

# 需要一个二维矩阵 x_train是三维的，要转成二维矩阵，再转成三维
# [None,28,28] ---> [None,784]
x_train_sacled = scalar.fit_transform(
x_train.astype(np.float32).reshape(-1,1)
).reshape(-1,28*28)

x_valid_scaled = scalar.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

x_test_scaled = scalar.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

y_train = np.asarray(y_train,dtype=np.int64)
y_valid = np.asarray(y_valid,dtype=np.int64)
y_test = np.asarray(y_test,dtype=np.int64)


In [11]:
np.max(x_train_sacled),np.min(x_train_sacled)

(2.0231433, -0.8105136)

In [12]:
def make_dataset(images,labels,epochs,batch_size,shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [13]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_sacled,y_train,epochs=epochs,batch_size=batch_size)

for data,label in dataset.take(1):
    print(data)
    print(label)

RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.

In [14]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_sacled,y_train,epochs=epochs,batch_size=batch_size)

# 1.auto initialization
# 2.不能被重新初始化
dataset_iter = dataset.make_one_shot_iterator()
x, y = dataset_iter.get_next()
with tf.Session() as sess:
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)


(20, 784)
(20,)


In [15]:
hidden_units=[100,100]
class_num =10

# x = tf.placeholder(dtype=tf.float32,shape=[None,28*28])
# y = tf.placeholder(dtype=tf.int64,shape=[None])

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer= tf.layers.dense(input_for_next_layer,hidden_unit,activation=tf.nn.relu)
    
logits = tf.layers.dense(input_for_next_layer,class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)

prediction = tf.argmax(logits,1)
correct_prediction = tf.equal(prediction,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [16]:
print(x)
print(logits)

Tensor("IteratorGetNext:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [17]:
# seesion
init = tf.global_variables_initializer()
# batch_size = 20
# epochs =10
train_steps_per_epoch = x_train.shape[0] // batch_size



with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
           
            loss_val,accuracy_val,_ = sess.run([loss,accuracy,train_op])
            print("\r[Train] epoch : %d,step: %d,loss: %3.5f,accuracy:%2.2f" % (epoch,step,loss_val,accuracy_val),end="")
        

[Train] epoch : 0,step: 0,loss: 2.42187,accuracy:0.20
[Train] epoch : 0,step: 1,loss: 2.17795,accuracy:0.30
[Train] epoch : 0,step: 2,loss: 1.99381,accuracy:0.30
[Train] epoch : 0,step: 3,loss: 2.13139,accuracy:0.25
[Train] epoch : 0,step: 4,loss: 1.82807,accuracy:0.40
[Train] epoch : 0,step: 5,loss: 1.62690,accuracy:0.40
[Train] epoch : 0,step: 6,loss: 1.45119,accuracy:0.55
[Train] epoch : 0,step: 7,loss: 1.37563,accuracy:0.60
[Train] epoch : 0,step: 8,loss: 1.41902,accuracy:0.50
[Train] epoch : 0,step: 9,loss: 1.82831,accuracy:0.35
[Train] epoch : 0,step: 10,loss: 1.06524,accuracy:0.65
[Train] epoch : 0,step: 11,loss: 1.42603,accuracy:0.45
[Train] epoch : 0,step: 12,loss: 1.56801,accuracy:0.45
[Train] epoch : 0,step: 13,loss: 1.05265,accuracy:0.65
[Train] epoch : 0,step: 14,loss: 1.21342,accuracy:0.55
[Train] epoch : 0,step: 15,loss: 1.12341,accuracy:0.55
[Train] epoch : 0,step: 16,loss: 0.84843,accuracy:0.75
[Train] epoch : 0,step: 17,loss: 1.12447,accuracy:0.65
[Train] epoch : 0,st

[Train] epoch : 0,step: 159,loss: 0.40447,accuracy:0.90
[Train] epoch : 0,step: 160,loss: 1.07040,accuracy:0.60
[Train] epoch : 0,step: 161,loss: 0.83599,accuracy:0.70
[Train] epoch : 0,step: 162,loss: 0.95085,accuracy:0.65
[Train] epoch : 0,step: 163,loss: 0.31871,accuracy:0.85
[Train] epoch : 0,step: 164,loss: 0.34590,accuracy:0.80
[Train] epoch : 0,step: 165,loss: 0.92007,accuracy:0.65
[Train] epoch : 0,step: 166,loss: 0.49517,accuracy:0.80
[Train] epoch : 0,step: 167,loss: 0.87045,accuracy:0.75
[Train] epoch : 0,step: 168,loss: 0.34291,accuracy:0.90
[Train] epoch : 0,step: 169,loss: 0.89496,accuracy:0.70
[Train] epoch : 0,step: 170,loss: 0.87460,accuracy:0.60
[Train] epoch : 0,step: 171,loss: 0.36215,accuracy:0.90
[Train] epoch : 0,step: 172,loss: 0.92382,accuracy:0.50
[Train] epoch : 0,step: 173,loss: 0.66206,accuracy:0.85
[Train] epoch : 0,step: 174,loss: 0.66648,accuracy:0.75
[Train] epoch : 0,step: 175,loss: 0.57736,accuracy:0.70
[Train] epoch : 0,step: 176,loss: 0.89945,accura

[Train] epoch : 0,step: 332,loss: 0.40978,accuracy:0.80
[Train] epoch : 0,step: 333,loss: 0.62879,accuracy:0.65
[Train] epoch : 0,step: 334,loss: 0.55793,accuracy:0.80
[Train] epoch : 0,step: 335,loss: 0.66820,accuracy:0.65
[Train] epoch : 0,step: 336,loss: 0.58350,accuracy:0.80
[Train] epoch : 0,step: 337,loss: 0.13428,accuracy:0.95
[Train] epoch : 0,step: 338,loss: 0.47715,accuracy:0.85
[Train] epoch : 0,step: 339,loss: 0.43050,accuracy:0.75
[Train] epoch : 0,step: 340,loss: 0.82416,accuracy:0.65
[Train] epoch : 0,step: 341,loss: 0.73710,accuracy:0.70
[Train] epoch : 0,step: 342,loss: 0.28381,accuracy:0.90
[Train] epoch : 0,step: 343,loss: 0.72466,accuracy:0.85
[Train] epoch : 0,step: 344,loss: 0.52559,accuracy:0.80
[Train] epoch : 0,step: 345,loss: 0.32890,accuracy:0.85
[Train] epoch : 0,step: 346,loss: 0.42980,accuracy:0.85
[Train] epoch : 0,step: 347,loss: 0.73885,accuracy:0.70
[Train] epoch : 0,step: 348,loss: 0.48671,accuracy:0.75
[Train] epoch : 0,step: 349,loss: 0.30902,accura

[Train] epoch : 0,step: 512,loss: 0.72151,accuracy:0.65
[Train] epoch : 0,step: 513,loss: 0.41284,accuracy:0.85
[Train] epoch : 0,step: 514,loss: 0.80986,accuracy:0.65
[Train] epoch : 0,step: 515,loss: 0.25799,accuracy:0.95
[Train] epoch : 0,step: 516,loss: 0.54382,accuracy:0.80
[Train] epoch : 0,step: 517,loss: 0.28069,accuracy:0.90
[Train] epoch : 0,step: 518,loss: 0.86472,accuracy:0.70
[Train] epoch : 0,step: 519,loss: 0.69018,accuracy:0.85
[Train] epoch : 0,step: 520,loss: 0.17547,accuracy:1.00
[Train] epoch : 0,step: 521,loss: 0.35587,accuracy:0.95
[Train] epoch : 0,step: 522,loss: 0.14986,accuracy:1.00
[Train] epoch : 0,step: 523,loss: 0.70170,accuracy:0.65
[Train] epoch : 0,step: 524,loss: 0.88764,accuracy:0.65
[Train] epoch : 0,step: 525,loss: 0.32569,accuracy:0.90
[Train] epoch : 0,step: 526,loss: 0.35273,accuracy:0.85
[Train] epoch : 0,step: 527,loss: 0.28657,accuracy:0.90
[Train] epoch : 0,step: 528,loss: 0.35819,accuracy:0.90
[Train] epoch : 0,step: 529,loss: 0.45104,accura

[Train] epoch : 0,step: 697,loss: 0.29835,accuracy:0.90
[Train] epoch : 0,step: 698,loss: 0.65596,accuracy:0.70
[Train] epoch : 0,step: 699,loss: 0.49220,accuracy:0.80
[Train] epoch : 0,step: 700,loss: 0.26186,accuracy:0.85
[Train] epoch : 0,step: 701,loss: 0.44791,accuracy:0.80
[Train] epoch : 0,step: 702,loss: 0.65204,accuracy:0.70
[Train] epoch : 0,step: 703,loss: 0.80095,accuracy:0.70
[Train] epoch : 0,step: 704,loss: 0.48528,accuracy:0.75
[Train] epoch : 0,step: 705,loss: 0.55998,accuracy:0.80
[Train] epoch : 0,step: 706,loss: 0.58316,accuracy:0.85
[Train] epoch : 0,step: 707,loss: 0.18166,accuracy:0.90
[Train] epoch : 0,step: 708,loss: 0.58437,accuracy:0.80
[Train] epoch : 0,step: 709,loss: 0.80682,accuracy:0.75
[Train] epoch : 0,step: 710,loss: 0.37552,accuracy:0.85
[Train] epoch : 0,step: 711,loss: 0.45651,accuracy:0.90
[Train] epoch : 0,step: 712,loss: 0.72676,accuracy:0.75
[Train] epoch : 0,step: 713,loss: 0.53472,accuracy:0.80
[Train] epoch : 0,step: 714,loss: 0.36125,accura

...

[Train] epoch : 9,step: 1750,loss: 0.17973,accuracy:0.95
[Train] epoch : 9,step: 1751,loss: 0.37070,accuracy:0.85
[Train] epoch : 9,step: 1752,loss: 0.11613,accuracy:0.95
[Train] epoch : 9,step: 1753,loss: 0.25489,accuracy:0.90
[Train] epoch : 9,step: 1754,loss: 0.12659,accuracy:0.95
[Train] epoch : 9,step: 1755,loss: 0.07103,accuracy:1.00
[Train] epoch : 9,step: 1756,loss: 0.44965,accuracy:0.80
[Train] epoch : 9,step: 1757,loss: 0.14440,accuracy:0.95
[Train] epoch : 9,step: 1758,loss: 0.09253,accuracy:0.95
[Train] epoch : 9,step: 1759,loss: 0.08895,accuracy:0.95
[Train] epoch : 9,step: 1760,loss: 0.29711,accuracy:0.90
[Train] epoch : 9,step: 1761,loss: 0.18488,accuracy:0.95
[Train] epoch : 9,step: 1762,loss: 0.05257,accuracy:1.00
[Train] epoch : 9,step: 1763,loss: 0.09869,accuracy:0.95
[Train] epoch : 9,step: 1764,loss: 0.24258,accuracy:0.90
[Train] epoch : 9,step: 1765,loss: 0.24750,accuracy:0.90
[Train] epoch : 9,step: 1766,loss: 0.13586,accuracy:0.95
[Train] epoch : 9,step: 1767,lo

[Train] epoch : 9,step: 1949,loss: 0.16822,accuracy:0.90
[Train] epoch : 9,step: 1950,loss: 0.18266,accuracy:0.90
[Train] epoch : 9,step: 1951,loss: 0.12798,accuracy:1.00
[Train] epoch : 9,step: 1952,loss: 0.30075,accuracy:0.85
[Train] epoch : 9,step: 1953,loss: 0.18498,accuracy:0.90
[Train] epoch : 9,step: 1954,loss: 0.05401,accuracy:1.00
[Train] epoch : 9,step: 1955,loss: 0.09129,accuracy:0.95
[Train] epoch : 9,step: 1956,loss: 0.21567,accuracy:0.95
[Train] epoch : 9,step: 1957,loss: 0.18072,accuracy:0.95
[Train] epoch : 9,step: 1958,loss: 0.18767,accuracy:0.85
[Train] epoch : 9,step: 1959,loss: 0.55401,accuracy:0.85
[Train] epoch : 9,step: 1960,loss: 0.18071,accuracy:0.95
[Train] epoch : 9,step: 1961,loss: 0.31710,accuracy:0.85
[Train] epoch : 9,step: 1962,loss: 0.31051,accuracy:0.80
[Train] epoch : 9,step: 1963,loss: 0.11669,accuracy:0.90
[Train] epoch : 9,step: 1964,loss: 0.06467,accuracy:0.95
[Train] epoch : 9,step: 1965,loss: 0.16571,accuracy:0.95
[Train] epoch : 9,step: 1966,lo

[Train] epoch : 9,step: 2151,loss: 1.09202,accuracy:0.75
[Train] epoch : 9,step: 2152,loss: 0.31086,accuracy:0.85
[Train] epoch : 9,step: 2153,loss: 0.24874,accuracy:0.90
[Train] epoch : 9,step: 2154,loss: 0.40288,accuracy:0.90
[Train] epoch : 9,step: 2155,loss: 0.16472,accuracy:0.95
[Train] epoch : 9,step: 2156,loss: 0.30892,accuracy:0.85
[Train] epoch : 9,step: 2157,loss: 0.39213,accuracy:0.80
[Train] epoch : 9,step: 2158,loss: 0.09467,accuracy:1.00
[Train] epoch : 9,step: 2159,loss: 0.39379,accuracy:0.80
[Train] epoch : 9,step: 2160,loss: 0.11149,accuracy:0.95
[Train] epoch : 9,step: 2161,loss: 0.16791,accuracy:0.95
[Train] epoch : 9,step: 2162,loss: 0.13721,accuracy:1.00
[Train] epoch : 9,step: 2163,loss: 0.34937,accuracy:0.85
[Train] epoch : 9,step: 2164,loss: 0.06909,accuracy:1.00
[Train] epoch : 9,step: 2165,loss: 0.10050,accuracy:0.95
[Train] epoch : 9,step: 2166,loss: 0.16666,accuracy:0.90
[Train] epoch : 9,step: 2167,loss: 0.28820,accuracy:0.80
[Train] epoch : 9,step: 2168,lo

[Train] epoch : 9,step: 2355,loss: 0.21138,accuracy:0.95
[Train] epoch : 9,step: 2356,loss: 0.32926,accuracy:0.90
[Train] epoch : 9,step: 2357,loss: 0.26656,accuracy:0.95
[Train] epoch : 9,step: 2358,loss: 0.14163,accuracy:0.95
[Train] epoch : 9,step: 2359,loss: 0.03204,accuracy:1.00
[Train] epoch : 9,step: 2360,loss: 0.05085,accuracy:1.00
[Train] epoch : 9,step: 2361,loss: 0.23183,accuracy:0.85
[Train] epoch : 9,step: 2362,loss: 0.42393,accuracy:0.85
[Train] epoch : 9,step: 2363,loss: 0.20061,accuracy:0.95
[Train] epoch : 9,step: 2364,loss: 0.23400,accuracy:0.95
[Train] epoch : 9,step: 2365,loss: 0.08820,accuracy:1.00
[Train] epoch : 9,step: 2366,loss: 0.44946,accuracy:0.85
[Train] epoch : 9,step: 2367,loss: 0.25559,accuracy:0.90
[Train] epoch : 9,step: 2368,loss: 0.48973,accuracy:0.80
[Train] epoch : 9,step: 2369,loss: 0.29438,accuracy:0.90
[Train] epoch : 9,step: 2370,loss: 0.18547,accuracy:0.90
[Train] epoch : 9,step: 2371,loss: 0.11427,accuracy:0.95
[Train] epoch : 9,step: 2372,lo

[Train] epoch : 9,step: 2561,loss: 0.17696,accuracy:0.90
[Train] epoch : 9,step: 2562,loss: 0.28815,accuracy:0.90
[Train] epoch : 9,step: 2563,loss: 0.04280,accuracy:1.00
[Train] epoch : 9,step: 2564,loss: 0.19785,accuracy:0.85
[Train] epoch : 9,step: 2565,loss: 0.12273,accuracy:0.95
[Train] epoch : 9,step: 2566,loss: 0.16289,accuracy:0.90
[Train] epoch : 9,step: 2567,loss: 0.48173,accuracy:0.80
[Train] epoch : 9,step: 2568,loss: 0.38398,accuracy:0.85
[Train] epoch : 9,step: 2569,loss: 0.27954,accuracy:0.90
[Train] epoch : 9,step: 2570,loss: 0.10485,accuracy:0.95
[Train] epoch : 9,step: 2571,loss: 0.35909,accuracy:0.85
[Train] epoch : 9,step: 2572,loss: 0.18153,accuracy:0.90
[Train] epoch : 9,step: 2573,loss: 0.23597,accuracy:0.90
[Train] epoch : 9,step: 2574,loss: 0.16437,accuracy:0.95
[Train] epoch : 9,step: 2575,loss: 0.08585,accuracy:1.00
[Train] epoch : 9,step: 2576,loss: 0.17879,accuracy:0.95
[Train] epoch : 9,step: 2577,loss: 0.14033,accuracy:0.95
[Train] epoch : 9,step: 2578,lo